In [3]:
import pandas as pd
import numpy as np

Draw = barrier
dyAvg = ave. speed rating over the horse over all its runs.
dyLTO = speed rating last time out
DSLR = days since last run
VDW = number of wins divided by win prize money (based on same race type for the this race)
Won P/L Before = Profit and Loss betting on the Betfair Exchange from
                    all races BEFORE the race you are looking at
B--M = Difference between the Breakfast Price and the Morning Price

In [4]:
cols = ['Race Time', 'Course', 'Distance (y)', 'Draw', 'Weight Rank', 'Breakfast Price', 'Morning Price',
        'SP Odds Decimal', 'Won (1=Won, 0=Lost)', 'Place (1=Placed, 0=UnPlaced)', 'Trainer/Jky Stats Rank',
        'WON SR Before', 'Position LTO', 'Horse Stats Rank', 'Proform Speed Rating']

In [5]:
data = pd.read_excel('raw_data/data.xls')

In [6]:
data.columns

Index(['Race Time', 'Course', 'Course Direction', 'Course Characteristics',
       'Distance', 'Prize', 'Distance (y)', 'Race Type', 'Runners', 'Class',
       ...
       'User Rating 3 Rank LTO', 'User Rating 3 Odds LTO', 'User Rating 4 LTO',
       'User Rating 4 Rank LTO', 'User Rating 4 Odds LTO', 'Speed Rating LTO',
       'PRC Average LTO', 'PRC Last Run LTO', 'PRC 2nd Last Run LTO',
       'PRC 3rd Last Run LTO'],
      dtype='object', length=214)

In [7]:
df = data[cols] \
    .fillna(0)
df['unique_id'] = df['Race Time'].astype(str) + '_' + df['Course']

df['unique_id_int'] = pd.factorize(df['unique_id'])[0]

df.head()

,Race Time,Course,Distance (y),Draw,Weight Rank,Breakfast Price,Morning Price,SP Odds Decimal,"Won (1=Won, 0=Lost)","Place (1=Placed, 0=UnPlaced)",Trainer/Jky Stats Rank,WON SR Before,Position LTO,Horse Stats Rank,Proform Speed Rating,unique_id,unique_id_int
0,2020-10-01 12:20:00,Salisbury,1760,2,1,10.0,11.0,13.0,0,1,1,0.0,0.0,1,61,2020-10-01 12:20:00_Salisbury,0
1,2020-10-01 12:20:00,Salisbury,1760,9,1,13.0,10.0,12.0,1,0,12,0.0,0.0,1,61,2020-10-01 12:20:00_Salisbury,0
2,2020-10-01 12:20:00,Salisbury,1760,12,1,11.0,11.0,17.0,0,0,8,0.0,4.0,9,51,2020-10-01 12:20:00_Salisbury,0
3,2020-10-01 12:20:00,Salisbury,1760,8,1,67.0,67.0,126.0,0,0,9,0.0,11.0,7,41,2020-10-01 12:20:00_Salisbury,0
4,2020-10-01 12:20:00,Salisbury,1760,10,1,21.0,19.0,21.0,0,0,3,0.0,0.0,1,37,2020-10-01 12:20:00_Salisbury,0


In [8]:
df.to_csv('clean_data/df1.csv')

In [9]:
df.dtypes

Race Time                       datetime64[ns]
Course                                  object
Distance (y)                             int64
Draw                                     int64
Weight Rank                              int64
Breakfast Price                        float64
Morning Price                          float64
SP Odds Decimal                        float64
Won (1=Won, 0=Lost)                      int64
Place (1=Placed, 0=UnPlaced)             int64
Trainer/Jky Stats Rank                   int64
WON SR Before                          float64
Position LTO                           float64
Horse Stats Rank                         int64
Proform Speed Rating                     int64
unique_id                               object
unique_id_int                            int64
dtype: object

In [10]:
from src.db_connection import DatabaseConnector

horse_data = 'horse_data_v1'

try:
    db_connector = DatabaseConnector()
    db_connector.connect()

    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {horse_data} (
        `Race Time` DATE,
        `Course` VARCHAR(255),  -- Replace with appropriate data type and length
        `Distance (y)` INT,
        `Draw` INT,
        `Weight Rank` INT,
        `Breakfast Price` FLOAT,
        `Morning Price` FLOAT,
        -- Add other columns similarly
        `unique_id` VARCHAR(255),
        `unique_id_int` INT
    )
    """

    db_connector.execute_query(create_table_query)

    insert_data_query = f"""
    INSERT INTO {horse_data} (`Race Time`, `Course`, `Distance (y)`, `Draw`, `Weight Rank`, `Breakfast Price`,
        `Morning Price`, `unique_id`, `unique_id_int`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    for index, row in df.iterrows():
        data_values = (row['Race Time'], row['Course'], row['Distance (y)'], row['Draw'], row['Weight Rank'],
                       row['Breakfast Price'], row['Morning Price'], row['unique_id'], row['unique_id_int'])
        db_connector.execute_query(insert_data_query, data_values)

    db_connector.disconnect()

    print(f'The data has been successfully saved to the database')

except ValueError as ve:
    print(f'Value error: {str(ve)}')
except Exception as e:
    print(f'An error occurred: {str(e)}')
  

Connected to the database
Disconnected from the database
The data has been successfully saved to the database


In [ ]:
try:
    db_connector = DatabaseConnector()
    db_connector.connect()

    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {horse_data} (
        `Race Time` DATE,
        `Course` VARCHAR(255),  -- Replace with appropriate data type and length
        `Distance (y)` INT,
        `Draw` INT,
        `Weight Rank` INT,
        `Breakfast Price` FLOAT,
        `Morning Price` FLOAT,
        -- Add other columns similarly
        `unique_id` VARCHAR(255),
        `unique_id_int` INT
    )
    """

    db_connector.execute_query(create_table_query)

    insert_data_query = f"""
    INSERT INTO {horse_data} (`Race Time`, `Course`, `Distance (y)`, `Draw`, `Weight Rank`, `Breakfast Price`,
        `Morning Price`, `unique_id`, `unique_id_int`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    for index, row in df.iterrows():
        data_values = (row['Race Time'], row['Course'], row['Distance (y)'], row['Draw'], row['Weight Rank'],
                       row['Breakfast Price'], row['Morning Price'], row['unique_id'], row['unique_id_int'])
        db_connector.execute_query(insert_data_query, data_values)

    db_connector.disconnect()

    print(f'The data has been successfully saved to the database')

except ValueError as ve:
    print(f'Value error: {str(ve)}')
except Exception as e:
    print(f'An error occurred: {str(e)}')